### Problem Set 3

who is the central player in this
technology area ? Who is the leader ? Who is an irrelevant actor ?

In [ ]:
import networkx as nx
import operator
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import community
from networkx.algorithms.community import k_clique_communities


In [ ]:
G = nx.read_edgelist('network_prof.txt')
       
# function that generate connected components as subgraphs:
def connected_component_subgraphs(G):
    for c in nx.connected_components(G):
        yield G.subgraph(c)

# check number of components of the network:

graph_max = max(connected_component_subgraphs(G))
graph_min = min(connected_component_subgraphs(G))

print("Information about the network: \n" + nx.info(G))
print("\nInformation about the max subgraph: \n" + nx.info(graph_max))
print("\nInformation about the min subgraph: \n" + nx.info(graph_min))

if G.number_of_nodes() == graph_max.number_of_nodes():
    # the number of nodes of the network is equal to the max subgraph. so there is only one subgraph.
    # we could have done the same with the min
    print("We only have one directed component")

In [3]:
nodes = list(G.nodes)
num_nodes = G.number_of_nodes()
position = nx.spring_layout(G) # position nodes using Fruchterman-Reingold force-directed algorithm.

In [14]:
# plot function
def plot_network(G, position, nodes, Z, color_special, new_figure):
    # G is the graph
    # position is calculated using Fruchterman-Reingold force-directed algorithm.
    # nodes: all nodes
    # Z: subset of nodes to highlight
    # color_special = color of these highlighted nodes
    # new_figure: start a new figure?
    
    if new_figure==True: plt.figure(figsize=(10,10))

    nx.draw_networkx_nodes(G, position, nodes, node_size = 7, node_color = 'y' , alpha = 0.7)
    nx.draw_networkx_nodes(G, position, Z, node_size = 14, node_color = color_special)

    nx.draw_networkx_edges(G, position, alpha = 0.5)
    plt.axis('off')
    if new_figure==True: plt.show()
    

In [15]:
# picture of the network:
plot_network(G, position, nodes, [], 'r', True)

## Significant Nodes
At this point we have to analyise our network. To understand the leader and irrelevant actors, we measure the vertex importance. We will analize

- Node degree
- Betweenness
- Eigenvector centrality

**Node Degree**: The node degree is the number of edges adjacent to the node

In [16]:
nodes_degree = G.degree
print(f" For example, node number '{list(nodes_degree)[0][0]}' degree is equal to {list(nodes_degree)[0][1]}.")

 For example, node number '0' degree is equal to 347.


In our dataset, each node is a person. Then the degree of that person is the number of people he/she is talking to. Let us see who are the people with more contacts:

In [17]:
degrees = [val for (node, val) in G.degree()] #only the values
nodes = [node for (node, val) in G.degree()] #only the node

Z = [x for _,x in sorted(zip(degrees,nodes), reverse=True)][0:10] # first ten nodes
degree_most_connected = G.degree(Z)
list(degree_most_connected)[:][1]

print("Nodes with highest DEGREE:")
for i in range(10):
    print(f"Position #{i+1}: node {list(degree_most_connected)[i][0]} with degree {list(degree_most_connected)[i][1]}")

Nodes with higest degree
Position #1 for degree: node 107 with degree 1045
Position #2 for degree: node 1684 with degree 792
Position #3 for degree: node 1912 with degree 755
Position #4 for degree: node 3437 with degree 547
Position #5 for degree: node 0 with degree 347
Position #6 for degree: node 2543 with degree 294
Position #7 for degree: node 2347 with degree 291
Position #8 for degree: node 1888 with degree 254
Position #9 for degree: node 1800 with degree 245
Position #10 for degree: node 1663 with degree 235


Highlight these nodes in the network:

Highlight nodes with the highest centrality


In [21]:
#PRINT

print("Highlight nodes with the highest centrality")
plot_network(G, position, nodes, Z, 'r', True)

**Betweenness**: measures how frequently a mode is in the shortest path between any two nodes

In [19]:
# warning!! the next command is realy slow

betweenness = nx.betweenness_centrality(G) # compute the shortest-path betweenness centrality for nodes. return a dictionary

{'0': 0.14630592147442917,
 '1': 2.7832744209034606e-06,
 '2': 7.595021178512074e-08,
 '3': 1.6850656559280464e-06,
 '4': 1.8403320547933104e-07,
 '5': 2.205964164092193e-06,
 '6': 2.4537760730577472e-08,
 '7': 0.0001702984836730339,
 '8': 2.7604980821899654e-07,
 '9': 1.6454236303026905e-05,
 '10': 4.986739552037655e-08,
 '11': 0.0,
 '12': 0.0,
 '13': 1.7622717578436846e-06,
 '14': 5.582871686568508e-07,
 '15': 0.0,
 '16': 1.9979459275532697e-07,
 '17': 4.1066669000480344e-07,
 '18': 0.0,
 '19': 5.062957964075819e-06,
 '20': 6.793693332142838e-07,
 '21': 0.0009380243844653233,
 '22': 6.703002200833232e-07,
 '23': 6.860348937590618e-06,
 '24': 1.3673472422981514e-07,
 '25': 5.38808313945586e-05,
 '26': 1.935436798204632e-05,
 '27': 3.067220091322184e-08,
 '28': 3.812160659244892e-07,
 '29': 1.3954817951917517e-06,
 '30': 1.3694627409316544e-06,
 '31': 4.932641252790837e-06,
 '32': 0.0,
 '33': 0.0,
 '34': 0.0036020881281963652,
 '35': 0.0,
 '36': 2.726417858953052e-08,
 '37': 0.0,
 '38'

We find the 10 people with highest betweenness. Betweenness centrality measures how important a node is to the shortest paths through the network. Hence, when we talk about links, it is important to find nodes which apprears more often.

In [22]:
btw = [val for (node, val) in betweenness] #only the values
nodes = [node for (node, val) in betweenness] #only the node

high_btw = [x for _,x in sorted(zip(btw,nodes), reverse=True)][0:10] # first ten nodes

print("Nodes with highest BETWEENNESS CENTRALITY")
for i in range(10):
    print(f"Position #{i+1}: node {list(degree_most_connected)[i][0]}")

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
# PRINT

print("Highlight nodes with the highest betweenness centrality")
plot_network(G, position, nodes, high_btw, 'g', True)


In [23]:
betweenness

{'0': 0.14630592147442917,
 '1': 2.7832744209034606e-06,
 '2': 7.595021178512074e-08,
 '3': 1.6850656559280464e-06,
 '4': 1.8403320547933104e-07,
 '5': 2.205964164092193e-06,
 '6': 2.4537760730577472e-08,
 '7': 0.0001702984836730339,
 '8': 2.7604980821899654e-07,
 '9': 1.6454236303026905e-05,
 '10': 4.986739552037655e-08,
 '11': 0.0,
 '12': 0.0,
 '13': 1.7622717578436846e-06,
 '14': 5.582871686568508e-07,
 '15': 0.0,
 '16': 1.9979459275532697e-07,
 '17': 4.1066669000480344e-07,
 '18': 0.0,
 '19': 5.062957964075819e-06,
 '20': 6.793693332142838e-07,
 '21': 0.0009380243844653233,
 '22': 6.703002200833232e-07,
 '23': 6.860348937590618e-06,
 '24': 1.3673472422981514e-07,
 '25': 5.38808313945586e-05,
 '26': 1.935436798204632e-05,
 '27': 3.067220091322184e-08,
 '28': 3.812160659244892e-07,
 '29': 1.3954817951917517e-06,
 '30': 1.3694627409316544e-06,
 '31': 4.932641252790837e-06,
 '32': 0.0,
 '33': 0.0,
 '34': 0.0036020881281963652,
 '35': 0.0,
 '36': 2.726417858953052e-08,
 '37': 0.0,
 '38'